In [1]:
# loops thru each first-round NFL playoff matchup
# adds quarterfinals to the EV calculation



import mysql.connector
from collections import defaultdict

conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# query to get Potential Payouts for Conference, Championship, and Quarterfinals events
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.EventType IN ('Conference Winner', 'Championship', 'Quarterfinals')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

payouts = defaultdict(lambda: {'payout_conference': 0, 'payout_championship': 0, 'payout_quarterfinals': 0})

# matchup structure with sample team names
matchups = {
    "AFC": [
        ("Denver Broncos", "Buffalo Bills"),
        ("Baltimore Ravens", "Pittsburgh Steelers"),
        ("Los Angeles Chargers", "Houston Texans")
    ],
    "NFC": [
        ("Green Bay Packers", "Philadelphia Eagles"),
        ("Washington Commanders", "Tampa Bay Buccaneers"),
        ("Minnesota Vikings", "Los Angeles Rams")
    ]
}

# probabilities and payouts for each team
team_probabilities = {
    "Denver Broncos": {'current_round_prob': 0.20, 'current_quarterfinals_prob': 0.08, 'current_conference_prob': 0.03, 'current_champ_prob': 0.01},
    "Buffalo Bills": {'current_round_prob': 0.80, 'current_quarterfinals_prob': 0.40, 'current_conference_prob': 0.24, 'current_champ_prob': 0.12},
    "Pittsburgh Steelers": {'current_round_prob': 0.18, 'current_quarterfinals_prob': 0.07, 'current_conference_prob': 0.02, 'current_champ_prob': 0.007},
    "Baltimore Ravens": {'current_round_prob': 0.82, 'current_quarterfinals_prob': 0.41,'current_conference_prob': 0.24, 'current_champ_prob': 0.12},
    "Los Angeles Chargers": {'current_round_prob': 0.57, 'current_quarterfinals_prob': 0.20,'current_conference_prob': 0.05, 'current_champ_prob': 0.02},
    "Houston Texans": {'current_round_prob': 0.43, 'current_quarterfinals_prob': 0.15, 'current_conference_prob': 0.02, 'current_champ_prob': 0.01},
    "Green Bay Packers": {'current_round_prob': 0.315, 'current_quarterfinals_prob': 0.14,'current_conference_prob': 0.06, 'current_champ_prob': 0.03},
    "Philadelphia Eagles": {'current_round_prob': 0.685, 'current_quarterfinals_prob': 0.34,'current_conference_prob': 0.20, 'current_champ_prob': 0.105},
    "Washington Commanders": {'current_round_prob': 0.395, 'current_quarterfinals_prob': 0.12,'current_conference_prob': 0.02, 'current_champ_prob': 0.01},
    "Tampa Bay Buccaneers": {'current_round_prob': 0.605, 'current_quarterfinals_prob': 0.16,'current_conference_prob': 0.06, 'current_champ_prob': 0.03},
    "Minnesota Vikings": {'current_round_prob': 0.545, 'current_quarterfinals_prob': 0.22,'current_conference_prob': 0.09, 'current_champ_prob': 0.04},
    "Los Angeles Rams": {'current_round_prob': 0.455, 'current_quarterfinals_prob': 0.18,'current_conference_prob': 0.045, 'current_champ_prob': 0.02}
}

# populating payouts based on fetched data
for participant_name, event_type, total_payout in cursor.fetchall():
    payouts[participant_name][f'payout_{event_type.lower().replace(" ", "_")}'] = float(total_payout)

cursor.close()
conn.close()

# defining EV calculation functions
def calculate_ev(probability, payout):
    return probability * payout

def calculate_conditional_ev(current_quarterfinals_prob, quarterfinals_payout, current_conference_prob, conference_payout, current_champ_prob, champ_payout, current_round_prob):
    conditional_quarterfinals_prob = current_quarterfinals_prob / current_round_prob
    conditional_conference_prob = current_conference_prob / current_round_prob
    conditional_champ_prob = current_champ_prob / current_round_prob
    
    quarterfinals_ev = calculate_ev(conditional_quarterfinals_prob, quarterfinals_payout)
    conference_ev = calculate_ev(conditional_conference_prob, conference_payout)
    champ_ev = calculate_ev(conditional_champ_prob, champ_payout)
    
    return quarterfinals_ev + conference_ev + champ_ev

# loop thru each conference and matchup, calculating EVs
for conference, games in matchups.items():
    print(f"\n{conference} Conference First-Round Matchups:\n")
    for team1_name, team2_name in games:
        team1 = {
            "current_round_prob": team_probabilities[team1_name]["current_round_prob"],
            "current_quarterfinals_prob": team_probabilities[team1_name]["current_quarterfinals_prob"],
            "current_conference_prob": team_probabilities[team1_name]["current_conference_prob"],
            "current_champ_prob": team_probabilities[team1_name]["current_champ_prob"],
            "quarterfinals_payout": payouts[team1_name]["payout_quarterfinals"],
            "conference_payout": payouts[team1_name]["payout_conference"],
            "champ_payout": payouts[team1_name]["payout_championship"]
        }

        team2 = {
            "current_round_prob": team_probabilities[team2_name]["current_round_prob"],
            "current_quarterfinals_prob": team_probabilities[team2_name]["current_quarterfinals_prob"],
            "current_conference_prob": team_probabilities[team2_name]["current_conference_prob"],
            "current_champ_prob": team_probabilities[team2_name]["current_champ_prob"],
            "quarterfinals_payout": payouts[team2_name]["payout_quarterfinals"],
            "conference_payout": payouts[team2_name]["payout_conference"],
            "champ_payout": payouts[team2_name]["payout_championship"]
        }

        # calculate conditional EVs
        team1_ev_win = calculate_conditional_ev(
            team1["current_quarterfinals_prob"], team1["quarterfinals_payout"],
            team1["current_conference_prob"], team1["conference_payout"],
            team1["current_champ_prob"], team1["champ_payout"],
            team1["current_round_prob"]
        )

        team2_ev_win = calculate_conditional_ev(
            team2["current_quarterfinals_prob"], team2["quarterfinals_payout"],
            team2["current_conference_prob"], team2["conference_payout"],
            team2["current_champ_prob"], team2["champ_payout"],
            team2["current_round_prob"]
        )

        # calculate Series Delta
        series_delta = abs(team1_ev_win - team2_ev_win)

        # outputting results
        print(f"{team1_name} vs {team2_name}")
        print(f"{team1_name} EV if win round: ${team1_ev_win:.2f}")
        print(f"{team2_name} EV if win round: ${team2_ev_win:.2f}")
        print(f"Series Delta: ${series_delta:.2f}\n")



AFC Conference First-Round Matchups:

Denver Broncos vs Buffalo Bills
Denver Broncos EV if win round: $25148.50
Buffalo Bills EV if win round: $10098.75
Series Delta: $15049.75

Baltimore Ravens vs Pittsburgh Steelers
Baltimore Ravens EV if win round: $9601.46
Pittsburgh Steelers EV if win round: $12725.81
Series Delta: $3124.34

Los Angeles Chargers vs Houston Texans
Los Angeles Chargers EV if win round: $2373.33
Houston Texans EV if win round: $151.16
Series Delta: $2222.17


NFC Conference First-Round Matchups:

Green Bay Packers vs Philadelphia Eagles
Green Bay Packers EV if win round: $10858.10
Philadelphia Eagles EV if win round: $10145.91
Series Delta: $712.18

Washington Commanders vs Tampa Bay Buccaneers
Washington Commanders EV if win round: $5824.56
Tampa Bay Buccaneers EV if win round: $19424.13
Series Delta: $13599.58

Minnesota Vikings vs Los Angeles Rams
Minnesota Vikings EV if win round: $10053.21
Los Angeles Rams EV if win round: $16026.37
Series Delta: $5973.16



In [2]:
# Playoffs Round/Series EV Calculator (Isolated to the Matchup)


import mysql.connector
from collections import defaultdict

conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# query to get Potential Payouts for Conf, Champ
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# payouts dictionary
payouts = defaultdict(lambda: {'payout_conference': 0, 'payout_championship': 0})

# define relevant teams by full names
relevant_teams = [
    
    ## INSERT FULL TEAM NAMES HERE
    'Detroit Tigers', 'Cleveland Guardians'
]

# organizing payouts by relevant teams
for participant_name, event_type, total_payout in cursor.fetchall():
    # Check if the participant name matches any relevant team
    if participant_name in relevant_teams:
        team_name = participant_name
        total_payout = float(total_payout)  # convert to float
        
        if event_type == 'Conference Winner':
            payouts[team_name]['payout_conference'] = total_payout
        elif event_type == 'Championship':
            payouts[team_name]['payout_championship'] = total_payout
        
        print(f"Team: {team_name}, Event Type: {event_type}, Total Payout: {total_payout}")

cursor.close()
conn.close()

# define EV calc functions
def calculate_ev(probability, payout):
    """Calculate expected value based on probability and payout."""
    return probability * payout

def calculate_conditional_ev(current_conference_prob, conference_payout, current_champ_prob, champ_payout, current_round_prob):
    """Calculate conditional EV if the team wins the current round."""
    conditional_conference_prob = current_conference_prob / current_round_prob
    conditional_champ_prob = current_champ_prob / current_round_prob
    conference_ev = calculate_ev(conditional_conference_prob, conference_payout)
    champ_ev = calculate_ev(conditional_champ_prob, champ_payout)
    return conference_ev + champ_ev

# inputs for the 2 teams
team1 = { 
    
    ## INSERT PROBABILITIES BELOW
    
    "current_round_prob": 0.534,      
    "current_conference_prob": 0.18,      
    "current_champ_prob": 0.06,        
    "conference_payout": payouts['Detroit Tigers']['payout_conference'],             
    "champ_payout": payouts['Detroit Tigers']['payout_championship']               
}

team2 = { 
    "current_round_prob": 1 - team1["current_round_prob"],      
    
     ## INSERT PROBABILITIES BELOW
    
    "current_conference_prob": 0.15,    
    "current_champ_prob": 0.06,       
    "conference_payout": payouts['Cleveland Guardians']['payout_conference'],  
    "champ_payout": payouts['Cleveland Guardians']['payout_championship']   
}

# calculate conditional EVs if each team wins current round
team1_ev_win = calculate_conditional_ev(
    team1["current_conference_prob"], team1["conference_payout"],
    team1["current_champ_prob"], team1["champ_payout"],
    team1["current_round_prob"]
)

team2_ev_win = calculate_conditional_ev(
    team2["current_conference_prob"], team2["conference_payout"],
    team2["current_champ_prob"], team2["champ_payout"],
    team2["current_round_prob"]
)

# calculate Series Delta as the diff. between conditional EVs
series_delta = abs(team1_ev_win - team2_ev_win)

print("Conditional Expected Values if Team Wins Current Round:")
print(f"Detroit Tigers EV if win round: ${team1_ev_win:.2f}")
print(f"Cleveland Guardians EV if win round: ${team2_ev_win:.2f}")
print("\nSeries Delta:")
print(f"${series_delta:.2f}")


Conditional Expected Values if Team Wins Current Round:
Detroit Tigers EV if win round: $0.00
Cleveland Guardians EV if win round: $0.00

Series Delta:
$0.00


In [4]:
## Playoffs Individual Game EV Calculator (Isolated to the Matchup)


def calculate_team_ev(new_series_odds, series_ev):
    """Calculate the team EV based on new series odds and series EV."""
    return new_series_odds * series_ev

def calculate_game3_ev_delta(team1_game3_win_prob, team2_game3_win_prob, team1_new_series_odds, team2_new_series_odds, team1_ev_win, team2_ev_win):
    """Calculate the EV Delta for Game 3."""
    
    # calculate EV if Team 1 wins Game 3
    team1_ev_if_win = calculate_team_ev(team1_new_series_odds, team1_ev_win)
    team2_ev_if_lose = calculate_team_ev(1 - team1_new_series_odds, team2_ev_win)
    ev_if_team1_wins_game3 = team1_ev_if_win + team2_ev_if_lose
    
    # calculate EV if Team 2 wins Game 3
    team1_ev_if_lose = calculate_team_ev(1 - team2_new_series_odds, team1_ev_win)
    team2_ev_if_win = calculate_team_ev(team2_new_series_odds, team2_ev_win)
    ev_if_team2_wins_game3 = team1_ev_if_lose + team2_ev_if_win
    
    # calculate EV Delta for Game 3
    ev_delta = abs(ev_if_team1_wins_game3 - ev_if_team2_wins_game3)
    
    return ev_if_team1_wins_game3, ev_if_team2_wins_game3, ev_delta

# inputs specific to Game 3
team1_game3_win_prob = 0.53  # Team 1's prob. of winning Game 3
team2_game3_win_prob = (1 - team1_game3_win_prob)  # Team 2's prob. of winning Game 3
team1_new_series_odds = 1  #series odds if Team 1 wins Game 3
team2_new_series_odds = 0.99148936170  #series odds if Team 2 wins Game 3

#use team1_ev_win and team2_ev_win from previous script
#calculate EV if Team 1 or Team 2 wins Game 3, and the EV Delta
ev_if_team1_wins_game3, ev_if_team2_wins_game3, game3_ev_delta = calculate_game3_ev_delta(
    team1_game3_win_prob,
    team2_game3_win_prob,
    team1_new_series_odds,
    team2_new_series_odds,
    team1_ev_win,  #from previous script
    team2_ev_win   #from previous script
)

print("Expected Values if either team wins Game 3:")
print(f"EV if Team 1 wins Game 3: ${ev_if_team1_wins_game3:.2f}")
print(f"EV if Team 2 wins Game 3: ${ev_if_team2_wins_game3:.2f}")
print("\nGame 3 EV Delta:")
print(f"EV Delta: ${game3_ev_delta:.2f}")


Expected Values if either team wins Game 3:
EV if Team 1 wins Game 3: $22972.13
EV if Team 2 wins Game 3: $7747.85

Game 3 EV Delta:
EV Delta: $15224.29


In [16]:
def calculate_ev(probability, payout):
    """Calculate expected value based on probability and payout."""
    return probability * payout

def calculate_conditional_ev(current_conference_prob, conference_payout, current_champ_prob, champ_payout, current_round_prob):
    """Calculate conditional EV if the team wins the current round."""
    conditional_conference_prob = current_conference_prob / current_round_prob
    conditional_champ_prob = current_champ_prob / current_round_prob
    conference_ev = calculate_ev(conditional_conference_prob, conference_payout)
    champ_ev = calculate_ev(conditional_champ_prob, champ_payout)
    return conference_ev + champ_ev

#Inputs for Tigers and Guardians
team1 = {  # Tigers
    "current_round_prob": 0.534,      
    "current_conference_prob": 0.18,      
    "current_champ_prob": 0.06,        
    "conference_payout": 0,             
    "champ_payout": 204452               
}

team2 = {  # Guardians
    "current_round_prob": 1 - team1["current_round_prob"],       
    "current_conference_prob": 0.15,      
    "current_champ_prob": 0.06,         
    "conference_payout": 0,     
    "champ_payout": 59160               
}

# calculate conditional EVs if each team wins current round
team1_ev_win = calculate_conditional_ev(
    team1["current_conference_prob"], team1["conference_payout"],
    team1["current_champ_prob"], team1["champ_payout"],
    team1["current_round_prob"]
)

team2_ev_win = calculate_conditional_ev(
    team2["current_conference_prob"], team2["conference_payout"],
    team2["current_champ_prob"], team2["champ_payout"],
    team2["current_round_prob"]
)

# calculate the Round Delta as the diff. between conditional EVs
series_delta = abs(team1_ev_win - team2_ev_win)

print("Conditional Expected Values if Team Wins Current Round:")
print(f"Tigers EV if win round: ${team1_ev_win:.2f}")
print(f"Guardians EV if win round: ${team2_ev_win:.2f}")
print("\nSeries Delta:")
print(f"Series Delta: ${series_delta:.2f}")


Conditional Expected Values if Team Wins Current Round:
Tigers EV if win round: $22972.13
Guardians EV if win round: $7617.17

Series Delta:
Series Delta: $15354.97


In [6]:
import mysql.connector
from collections import defaultdict

conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# query to get Potential Payouts for Conference and Championship events
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

payouts = defaultdict(lambda: {'payout_conference': 0, 'payout_championship': 0})

#matchup structure with sample team names
matchups = {
    "AFC": [
        ("Kansas City Chiefs", "Miami Dolphins"),
        ("Cincinnati Bengals", "Pittsburgh Steelers"),
        ("Jacksonville Jaguars", "Cleveland Browns")
    ],
    "NFC": [
        ("San Francisco 49ers", "Minnesota Vikings"),
        ("Detroit Lions", "Atlanta Falcons"),
        ("Dallas Cowboys", "Seattle Seahawks")
    ]
}

# define probs. and payouts for each team
team_probabilities = {
    "Kansas City Chiefs": {'current_round_prob': 0.75, 'current_conference_prob': 0.2, 'current_champ_prob': 0.1},
    "Miami Dolphins": {'current_round_prob': 0.25, 'current_conference_prob': 0.1, 'current_champ_prob': 0.05},
    "Cincinnati Bengals": {'current_round_prob': 0.7, 'current_conference_prob': 0.18, 'current_champ_prob': 0.08},
    "Pittsburgh Steelers": {'current_round_prob': 0.3, 'current_conference_prob': 0.08, 'current_champ_prob': 0.04},
    "Jacksonville Jaguars": {'current_round_prob': 0.6, 'current_conference_prob': 0.15, 'current_champ_prob': 0.07},
    "Cleveland Browns": {'current_round_prob': 0.4, 'current_conference_prob': 0.09, 'current_champ_prob': 0.03},
    "San Francisco 49ers": {'current_round_prob': 0.75, 'current_conference_prob': 0.22, 'current_champ_prob': 0.12},
    "Minnesota Vikings": {'current_round_prob': 0.25, 'current_conference_prob': 0.11, 'current_champ_prob': 0.05},
    "Detroit Lions": {'current_round_prob': 0.65, 'current_conference_prob': 0.17, 'current_champ_prob': 0.07},
    "Atlanta Falcons": {'current_round_prob': 0.35, 'current_conference_prob': 0.08, 'current_champ_prob': 0.03},
    "Dallas Cowboys": {'current_round_prob': 0.6, 'current_conference_prob': 0.18, 'current_champ_prob': 0.08},
    "Seattle Seahawks": {'current_round_prob': 0.4, 'current_conference_prob': 0.09, 'current_champ_prob': 0.04}
}

# populate payouts based on fetched data
for participant_name, event_type, total_payout in cursor.fetchall():
    payouts[participant_name][f'payout_{event_type.lower().replace(" ", "_")}'] = float(total_payout)

cursor.close()
conn.close()

# def EV calculation functions
def calculate_ev(probability, payout):
    return probability * payout

def calculate_conditional_ev(current_conference_prob, conference_payout, current_champ_prob, champ_payout, current_round_prob):
    conditional_conference_prob = current_conference_prob / current_round_prob
    conditional_champ_prob = current_champ_prob / current_round_prob
    conference_ev = calculate_ev(conditional_conference_prob, conference_payout)
    champ_ev = calculate_ev(conditional_champ_prob, champ_payout)
    return conference_ev + champ_ev

# loop through each conference and matchup, calculating EVs
for conference, games in matchups.items():
    print(f"\n{conference} Conference First-Round Matchups:\n")
    for team1_name, team2_name in games:
        team1 = {
            "current_round_prob": team_probabilities[team1_name]["current_round_prob"],
            "current_conference_prob": team_probabilities[team1_name]["current_conference_prob"],
            "current_champ_prob": team_probabilities[team1_name]["current_champ_prob"],
            "conference_payout": payouts[team1_name]["payout_conference"],
            "champ_payout": payouts[team1_name]["payout_championship"]
        }

        team2 = {
            "current_round_prob": team_probabilities[team2_name]["current_round_prob"],
            "current_conference_prob": team_probabilities[team2_name]["current_conference_prob"],
            "current_champ_prob": team_probabilities[team2_name]["current_champ_prob"],
            "conference_payout": payouts[team2_name]["payout_conference"],
            "champ_payout": payouts[team2_name]["payout_championship"]
        }

        # Calculate conditional EVs
        team1_ev_win = calculate_conditional_ev(
            team1["current_conference_prob"], team1["conference_payout"],
            team1["current_champ_prob"], team1["champ_payout"],
            team1["current_round_prob"]
        )

        team2_ev_win = calculate_conditional_ev(
            team2["current_conference_prob"], team2["conference_payout"],
            team2["current_champ_prob"], team2["champ_payout"],
            team2["current_round_prob"]
        )

        # Calculate Series Delta
        series_delta = abs(team1_ev_win - team2_ev_win)

        print(f"{team1_name} vs {team2_name}")
        print(f"{team1_name} EV if win round: ${team1_ev_win:.2f}")
        print(f"{team2_name} EV if win round: ${team2_ev_win:.2f}")
        print(f"Series Delta: ${series_delta:.2f}\n")



AFC Conference First-Round Matchups:

Kansas City Chiefs vs Miami Dolphins
Kansas City Chiefs EV if win round: $14369.33
Miami Dolphins EV if win round: $0.00
Series Delta: $14369.33

Cincinnati Bengals vs Pittsburgh Steelers
Cincinnati Bengals EV if win round: $8564.57
Pittsburgh Steelers EV if win round: $25804.67
Series Delta: $17240.10

Jacksonville Jaguars vs Cleveland Browns
Jacksonville Jaguars EV if win round: $2628.50
Cleveland Browns EV if win round: $0.00
Series Delta: $2628.50


NFC Conference First-Round Matchups:

San Francisco 49ers vs Minnesota Vikings
San Francisco 49ers EV if win round: $0.00
Minnesota Vikings EV if win round: $10595.00
Series Delta: $10595.00

Detroit Lions vs Atlanta Falcons
Detroit Lions EV if win round: $19313.00
Atlanta Falcons EV if win round: $0.00
Series Delta: $19313.00

Dallas Cowboys vs Seattle Seahawks
Dallas Cowboys EV if win round: $4506.67
Seattle Seahawks EV if win round: $6133.50
Series Delta: $1626.83

